In [1]:
from coveopush import CoveoPush
from coveopush import Document

from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json

In [8]:
URL = "https://pokemondb.net/pokedex/national"
POKEDEX_ENDPOINT = 'https://pokemondb.net'

page = requests.get(URL)

In [3]:
soup = BeautifulSoup(page.content, "html.parser")

In [4]:
pokemon_names = soup.find_all(class_='ent-name')

In [5]:
# pokemon link
bublbasaur_link = pokemon_names[0]['href']

# pokemon name
pokemon_name = pokemon_names[0].get_text()

POKEDEX_ENDPOINT = 'https://pokemondb.net'
bublbasaur_page = requests.get(POKEDEX_ENDPOINT + bublbasaur_link)
bublbasaur_soup = BeautifulSoup(bublbasaur_page.content, "html.parser")

In [7]:
bublbasaur_data = bublbasaur_soup.find('tr'),{"class":"vitals-table"}
bublbasaur_soup.find("th", text="Abilities").next_sibling.next_sibling.a.text

'Overgrow'

In [8]:
# pokemon image
bublbasaur_img = bublbasaur_soup.find('img')['src']
bublbasaur_img

'https://img.pokemondb.net/artwork/bulbasaur.jpg'

In [9]:
bublbasaur_data = bublbasaur_soup.find('table', class_='vitals-table')

# pokemon national number
bulbasaur_num = bublbasaur_data.find("th", text="National №").find_next('td').get_text()

# pokemon type(s)
bublbasaur_types = bublbasaur_data.find_all(class_='type-icon')
types = list(t.get_text() for t in bublbasaur_types)

# type1 = bublbasaur_types[0].get_text()
# if len(bublbasaur_types) == 2:
#     type2 = bublbasaur_types[1].get_text()
# else:
#     type2 = '-'

# pokemon species
pokemon_species = bublbasaur_soup.find('th', text='Species').find_next('td').get_text()

# pokemon height 
pokemon_height = bublbasaur_data.find('th', text='Height').find_next('td').get_text().split()[0]

# pokemon weight 
pokemon_weight = bublbasaur_data.find('th', text='Weight').find_next('td').get_text().split()[0]

# pokemon abilities
pokemon_abilities_html = bublbasaur_data.find('th', text='Abilities').find_next('td').find_all('a')
pokemon_abilities = list(a.get_text() for a in pokemon_abilities_html)


In [42]:
# create dict with pokemon data
bublbasaur = {
    'name': pokemon_name,
    'link': bublbasaur_link,
    'img': bublbasaur_img,
    'nat_num': bulbasaur_num,
    'types': types,
    'species': pokemon_species,
    'height': pokemon_height,
    'weight': pokemon_weight,
    'abilities': pokemon_abilities
}

pokemon_json = json.dumps(bublbasaur)
pokemon_json

'{"name": "Bulbasaur", "link": "/pokedex/bulbasaur", "img": "https://img.pokemondb.net/artwork/bulbasaur.jpg", "nat_num": "001", "types": ["Grass", "Poison"], "species": "Seed Pok\\u00e9mon", "height": "0.7", "weight": "6.9", "abilities": ["Overgrow", "Chlorophyll"]}'

In [63]:
src_id = 'agongcoveocomgrotesquechinchillaxx1ef20h-vlfshqcn5gn3uuvbimpggxcnya'
org_id = 'agongcoveocomgrotesquechinchillaxx1ef20h'
api_key = 'xx2c315919-8534-4813-b884-389e31db0ad1'

push = CoveoPush.Push(src_id, org_id, api_key)
push.Start()

# meta_keys = ['name', 'link', 'img', 'nat_num', 'types', 'species', 'height', 'weight', 'abilities']

# bublbasaur_object = json.load(pokemon_json)
my_doc = Document(POKEDEX_ENDPOINT + bublbasaur_link)
my_doc.SetData(requests.get(POKEDEX_ENDPOINT + bublbasaur_link).text)

for k in bublbasaur.keys():
    my_doc.AddMetadata(k, bublbasaur[k])

push.AddSingleDocument(my_doc)

In [77]:
generations_html = soup.find_all('h2')

pokemon_generation = generations_html[0].text.split()[1]

info_cards = pokemon_generation.find_n


'1'

In [79]:
bublbasaur_soup.find_all('abbr')[0].text

'Generation 1'

In [124]:
def format_name(pokemon_name):
    return ''.join(pokemon_name.split()).lower()

In [5]:
def get_initial_infos(ref):
    # Get pokemon link
    link = ref['href']

    # Get pokemon name
    name = ref.text

    return link, name


In [117]:
def get_general_data_tab(child_name, child_soup):
    # Retrieve Pokemon's image
    pokemon_img = child_soup.find('img')['src']
    
    # Retrieve Pokemon's 'Pokedex data' table
    pokemon_data = child_soup.find('table', class_='vitals-table')

    # pokemon type(s)
    pokemon_types_html = pokemon_data.find_all(class_='type-icon')
    pokemon_types = list(t.text for t in pokemon_types_html)

    # pokemon height 
    pokemon_height = pokemon_data.find('th', text='Height').find_next('td').get_text().split()[0]

    # pokemon weight 
    pokemon_weight = pokemon_data.find('th', text='Weight').find_next('td').get_text().split()[0]

    # pokemon abilities
    pokemon_abilities_html = pokemon_data.find('th', text='Abilities').find_next('td').find_all('a')
    pokemon_abilities = list(a.text for a in pokemon_abilities_html)

    # create dict with pokemon data
    pokemon = {
        'name': child_name,
        'img': pokemon_img,
        # 'desc': pokemon_desc,
        # 'nat_num': pokemon_num,
        'types': pokemon_types,
        'height': pokemon_height,
        'weight': pokemon_weight,
        'abilities': pokemon_abilities,
        'folding_child': format_name(child_name)
    }

    return pokemon

In [135]:
def get_general_data(pokemon_name, pokemon_link):
    # Create individual pokemon's soup
    pokemon_page = requests.get(POKEDEX_ENDPOINT + pokemon_link)
    pokemon_soup = BeautifulSoup(pokemon_page.content, "html.parser")

    # Retrieve Pokemon's image
    pokemon_img = pokemon_soup.find('img')['src']

    # Retrieve Pokemon's generation
    pokemon_generation = pokemon_soup.find('abbr').text

    # Retrieve short description 
    pokemon_desc = pokemon_soup.find('p').text
    
    # Retrieve Pokemon's 'Pokedex data' table
    pokemon_data = pokemon_soup.find('table', class_='vitals-table')

    # pokemon national number
    pokemon_num = pokemon_data.find("th", text="National №").find_next('td').get_text()

    # pokemon type(s)
    pokemon_types_html = pokemon_data.find_all(class_='type-icon')
    pokemon_types = list(t.text for t in pokemon_types_html)

    # pokemon species
    pokemon_species = pokemon_soup.find('th', text='Species').find_next('td').get_text()

    # pokemon height 
    pokemon_height = pokemon_data.find('th', text='Height').find_next('td').get_text().split()[0]

    # pokemon weight 
    pokemon_weight = pokemon_data.find('th', text='Weight').find_next('td').get_text().split()[0]

    # pokemon abilities
    pokemon_abilities_html = pokemon_data.find('th', text='Abilities').find_next('td').find_all('a')
    pokemon_abilities = list(a.text for a in pokemon_abilities_html)

    # Format pokemon's name
    formatted_name = format_name(pokemon_name)

    pokemon_forms = []
    
    # create dict with pokemon data
    pokemon = {
        'name': pokemon_name,
        'link': pokemon_link,
        'generation': pokemon_generation,
        'img': pokemon_img,
        'desc': pokemon_desc,
        'nat_num': pokemon_num,
        'types': pokemon_types,
        'species': pokemon_species,
        'height': pokemon_height,
        'weight': pokemon_weight,
        'abilities': pokemon_abilities,
        'folding_collection': formatted_name,
        'folding_parent': formatted_name,
        'folding_child': formatted_name
    }

    pokemon_forms.append(pokemon)

    # Retrieve Pokemon's other forms
    pokemon_tabs = pokemon_soup.find_all(class_='sv-tabs-tab', href=re.compile(r'#tab-basic'))
    pokemon_special_forms = pokemon_tabs[1:]

    if pokemon_special_forms:
        for p in pokemon_special_forms:
            child_name = p.text
            child_soup = pokemon_soup.find('div', id=p['href'][1:])
            child_obj = get_general_data_tab(child_name, child_soup)
            child_obj['folding_collection'] = formatted_name
            child_obj['folding_parent'] = formatted_name
            child_obj['link'] = pokemon_link + p['href']
            pokemon_forms.append(child_obj)

    return pokemon_forms


In [136]:
get_general_data('Latios', '/pokedex/latios')

[{'name': 'Latios',
  'link': '/pokedex/latios',
  'generation': 'Generation 3',
  'img': 'https://img.pokemondb.net/artwork/latios.jpg',
  'desc': 'Latios is a Dragon/Psychic type Pokémon introduced in Generation 3. It is known as the Eon Pokémon.',
  'nat_num': '381',
  'types': ['Dragon', 'Psychic'],
  'species': 'Eon Pokémon',
  'height': '2.0',
  'weight': '60.0',
  'abilities': ['Levitate'],
  'folding_collection': 'latios',
  'folding_parent': 'latios',
  'folding_child': 'latios'},
 {'name': 'Mega Latios',
  'img': 'https://img.pokemondb.net/artwork/latios-mega.jpg',
  'types': ['Dragon', 'Psychic'],
  'height': '2.3',
  'weight': '70.0',
  'abilities': ['Levitate'],
  'folding_child': 'megalatios',
  'folding_collection': 'latios',
  'folding_parent': 'latios',
  'link': '/pokedex/latios#tab-basic-11048'}]

In [137]:
# Load national pokedex page
URL = "https://pokemondb.net/pokedex/national"
POKEDEX_ENDPOINT = 'https://pokemondb.net'

page = requests.get(URL)

# Cresate initial page soup
soup = BeautifulSoup(page.content, "html.parser")

# Retrieve all Pokemon refs
pokemon_refs = soup.find_all(class_='ent-name')

all_pokemons = []

for r in pokemon_refs:
    pokemon_link, pokemon_name = get_initial_infos(r)
    pokemon_obj = get_general_data(pokemon_name, pokemon_link)
    
    all_pokemons.extend(pokemon_obj)

# Start using Push API
src_id = 'agongcoveocomgrotesquechinchillaxx1ef20h-vlfshqcn5gn3uuvbimpggxcnya'
org_id = 'agongcoveocomgrotesquechinchillaxx1ef20h'
api_key = 'xx2c315919-8534-4813-b884-389e31db0ad1'

push = CoveoPush.Push(src_id, org_id, api_key)
push.Start()

for p in all_pokemons:
    pokemon_page_link = POKEDEX_ENDPOINT + p['link']
    my_doc = Document(pokemon_page_link)
    my_doc.SetData(requests.get(pokemon_page_link).text)

    for k in p.keys():
        my_doc.AddMetadata(k, p[k])
    push.Add(my_doc)

push.End(True, True)